# Methods

In [1]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np
from scipy import interpolate
import math
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/cl/1j2m5n5d6tz7j370j52_b9v00000gn/T/ipykernel_74276/3777615979.py:1: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [3]:
init_notebook_mode(connected=True)

In [4]:
def TLARstatus(x, dico):
    for k in [key for key in isoDict.keys() if 'tlar' in key]:  #for k in dico.keys():
        if np.isnan(x[dico[k]['TLAR']]):
            return False
        else:
            if dico[k]['crit'] == '>':
                if x[dico[k]['TLAR']] < dico[k]['TLARtgt']:
                    return False
            else:
                if x[dico[k]['TLAR']] > dico[k]['TLARtgt']:
                    return False
    return True



def compute_isoTLAR(dico):
    TLAR = dico['TLAR']
    TLARtgt = dico['TLARtgt']
    TLARe= []
    TLARw = []
    
    if 'filter' in dico.keys(): #TODO: improve to be more generic with regards what dico['filter']['crit'] can be (use of eval function tbc)
        if dico['filter']['crit'] == '>':
            DOEflt = DOE[DOE[dico['filter']['by']] >=  dico['filter']['TLARtgt']]
        else:
            DOEflt = DOE[DOE[dico['filter']['by']] <=  dico['filter']['TLARtgt']]
    else:
        DOEflt = DOE.copy()

    for S in DOEflt['Total_Area'].unique():
        TLARw.append(S)
        filt = DOEflt[DOEflt['Total_Area']==S]
        
        #f = interpolate.interp1d(filt[TLAR], filt['MTO_AET'], bounds_error=False, fill_value="extrapolate")
        f = interpolate.interp1d(filt[TLAR], filt['MTO_AET'], bounds_error=True)
        try:
            TLARe.append(float(f(TLARtgt)))
        except:
            TLARe.append(np.nan)
    

    for k in DOEflt['MTO_AET'].unique():
        TLARe.append(k)
        filt = DOEflt[DOEflt['MTO_AET']==k]
        try:
            #f1 = interpolate.interp1d(filt[TLAR], filt['Total_Area'], bounds_error=False, fill_value="extrapolate")
            f1 = interpolate.interp1d(filt[TLAR], filt['Total_Area'], bounds_error=True)
            TLARw.append(float(f1(TLARtgt)))
        except:
            TLARw.append(np.nan)
        #TLARw.append(float(f1(TLARtgt)))


    isoTLAR = pd.DataFrame({'Total_Area': TLARw, 'MTO_AET': TLARe})

    isoTLAR = isoTLAR[ (isoTLAR.MTO_AET <= DOEflt['MTO_AET'].max()) & \
                      (isoTLAR.MTO_AET >= DOEflt['MTO_AET'].min()) & \
                      (isoTLAR.Total_Area <= DOEflt['Total_Area'].max()) & \
                      (isoTLAR.Total_Area >= DOEflt['Total_Area'].min())]
    isoTLAR = isoTLAR.sort_values(by=['Total_Area'])
    return isoTLAR

def compute_isoTLARmcl(dico, mcl):
    TLAR = dico['TLAR']
    TLARtgt = dico['TLARtgt']
    TLARe= []
    TLARw = []
    
    if 'filter' in dico.keys(): #TODO: improve to be more generic with regards what dico['filter']['crit'] can be (use of eval function tbc)
        if dico['filter']['crit'] == '>':
            DOEflt = DOE[(DOE[dico['filter']['by']] >=  dico['filter']['TLARtgt'])]
        else:
            DOEflt = DOE[(DOE[dico['filter']['by']] <=  dico['filter']['TLARtgt'])]
    else:
        DOEflt = DOE.copy()
    
    DOEflt = DOEflt[DOEflt.MCL==mcl]

    for S in DOEflt['Total_Area'].unique():
        TLARw.append(S)
        filt = DOEflt[DOEflt['Total_Area']==S]
        
        #f = interpolate.interp1d(filt[TLAR], filt['MTO_AET'], bounds_error=False, fill_value="extrapolate")
        f = interpolate.interp1d(filt[TLAR], filt['MTO_AET'], bounds_error=True)
        try:
            TLARe.append(float(f(TLARtgt)))
        except:
            TLARe.append(np.nan)
    

    for k in DOEflt['MTO_AET'].unique():
        TLARe.append(k)
        filt = DOEflt[DOEflt['MTO_AET']==k]
        try:
            #f1 = interpolate.interp1d(filt[TLAR], filt['Total_Area'], bounds_error=False, fill_value="extrapolate")
            f1 = interpolate.interp1d(filt[TLAR], filt['Total_Area'], bounds_error=True)
            TLARw.append(float(f1(TLARtgt)))
        except:
            TLARw.append(np.nan)
        #TLARw.append(float(f1(TLARtgt)))


    isoTLAR = pd.DataFrame({'Total_Area': TLARw, 'MTO_AET': TLARe})

    isoTLAR = isoTLAR[ (isoTLAR.MTO_AET <= DOEflt['MTO_AET'].max()) & \
                      (isoTLAR.MTO_AET >= DOEflt['MTO_AET'].min()) & \
                      (isoTLAR.Total_Area <= DOEflt['Total_Area'].max()) & \
                      (isoTLAR.Total_Area >= DOEflt['Total_Area'].min())]
    isoTLAR = isoTLAR.sort_values(by=['Total_Area'])
    return isoTLAR


def get_value_at(param, kfn, Sref):
    param_interp = []
    for S in np.sort(DOE['Total_Area'].unique()):
        filt = DOE[DOE['Total_Area']==S]
        fa = interpolate.interp1d(filt['MTO_AET'], filt[param])
        param_interp.append(fa(kfn))
    fb = interpolate.interp1d(np.sort(DOE['Total_Area'].unique()), param_interp)
    return fb(Sref)

def getMultiLinInterp(df, inputParam):
    #inputParam: list of input parameters we want interpolate into 
    interp_dict = {}
    for p in DOE.columns:
        if (p not in inputParam) and (df[p].dtypes in ['float64', 'float32', 'int64', 'int32']):
            interp_dict[p] = interpolate.LinearNDInterpolator(df[inputParam].values.tolist(), df[p], rescale=True)
    return interp_dict

In [5]:
def carpetplot(param, unit='', L = [], smooth = 0, cheaterslope=None):
    dfDOE=DOE[DOE[param].notna()]
    trace0 = go.Carpet(
        a = dfDOE['Total_Area'],
        b = dfDOE['MTO_AET'],
        y = dfDOE[param],
        cheaterslope = cheaterslope,
        aaxis = dict(
            #tickprefix = 'Sref = ',
            #ticksuffix = 'm²',
            smoothing = smooth,
            minorgridcount = 4, #24,
            gridcolor = 'black',
            minorgridcolor = "rgb(230, 230, 230)",
            title = "Total area (m²)"
        ),
        baxis = dict(
            #tickprefix = 'MTO_AET = ',
            #ticksuffix = '',
            smoothing = smooth,
            minorgridcount = 3, #24,
            gridcolor = 'black',
            minorgridcolor = "rgb(230, 230, 230)",
            title = "MTO_AET (lbf)"
        )
    )

    traceOK = go.Scattercarpet(
        a = dfDOE[dfDOE.status==True]['Total_Area'],
        b = dfDOE[dfDOE.status==True]['MTO_AET'],
        showlegend = False,
        mode = "markers",
        marker = dict(
            color = "rgb(102, 255, 51)"
            ),
        name = ''
    )

    traceKO = go.Scattercarpet(
        a = dfDOE[dfDOE.status==False]['Total_Area'],
        b = dfDOE[dfDOE.status==False]['MTO_AET'],
        showlegend = False,
        mode = "markers",
        marker = dict(
            color = 'red',
            symbol = 'circle'
            ),
        name = ''
    )
    
    data = [trace0, traceOK, traceKO]

    if 'OPT_CONFIG' in dfDOE.columns : 
        traceConf = go.Scattercarpet(
            a = dfDOE['Total_Area'],
            b = dfDOE['MTO_AET'],
            showlegend = False,
            mode = "text",
            text = dfDOE['OPT_CONFIG'],
            textposition = "top right",
            name = ''
        )
        data += traceConf
    
    if len(L)>0:
        col = []
        for l in L:
            try:
                col.append(l[2])
            except:
                col.append('blue')
        data.append(go.Scattercarpet(
            a = np.array([l[0] for l in L]),
            b = np.array([l[1] for l in L]),
            showlegend = False,
            mode = "markers",
            marker = dict(
                color = col, #np.array([l[2] for l in L]),
                symbol = "star-square-open-dot",
                size = 12
                ),
            name = ''
    )
        )


    for k in isoDict.keys():
        if isoDict[k]['color'] != False:
            isoTLAR = compute_isoTLAR(isoDict[k])
            trace = go.Scattercarpet(
                a = isoTLAR['Total_Area'],
                b = isoTLAR['MTO_AET'],
                mode = 'lines',
                line = dict(
                  shape = 'spline',
                  smoothing = 1,
                  color = isoDict[k]['color'],
                  dash = isoDict[k]['linetype']
                ),
                name = isoDict[k]['TLAR'] + ': ' + str(isoDict[k]['TLARtgt'])
              )
            data.append(trace)

    margin=(dfDOE[param].max()-dfDOE[param].min())/10
    
    layout = go.Layout(
        width=640,
        height=310,
        margin=dict(t=5, b=5, l=60, r=5),
        paper_bgcolor="LightSteelBlue",
        plot_bgcolor ='rgba(250,250,250,1)',
        #title = dict(
            #text='Carpet plot - {}'.format(param)),
        yaxis = dict(
            visible=True,
            title = dict(
                text= param +' '+ unit),
            #nticks = 30 # to be improved,
            range=[math.floor((dfDOE[param].min()-1.8*margin)/10)*10, math.floor((dfDOE[param].max()+margin)/10)*10],
            gridcolor='rgba(220,220,220,0.8)',
            gridwidth=1.2
            ),        
        xaxis = dict(
            visible=False,
            range=[-0.075, 1.075]
            ),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01)
        )

    fig = go.Figure(data = data, layout=layout)

    iplot(fig)

In [6]:
def carpetplot3D(param, unit='', L = [], chSlope=1, chart_title=None):
    dfDOE=DOE[DOE[param].notna()]
    fig = go.Figure()
    
    #dataA = dfDOE[dfDOE.MCL==dfDOE.MCL.min()]['Total_Area'].astype(float)
    #dataB = dfDOE[dfDOE.MCL==dfDOE.MCL.min()]['MTO_AET'].astype(float)
    dataA = dfDOE['Total_Area'].astype(float)
    dataB = dfDOE['MTO_AET'].astype(float)
    slpA = (1-float(chSlope)/(1+chSlope))/(dataA.max()-dataA.min())
    slpB = float(chSlope)/(1+chSlope)/(dataB.max()-dataB.min())
    #dataX= dataA*slpA - dataB*slpB
    #dataXnorm = ((dataX - dataX.min())/(dataX.max()-dataX.min()))
    dfDOE['dataX']=dataA*slpA - dataB*slpB
    dfDOE['dataXnorm']=((dfDOE.dataX - dfDOE.dataX.min())/(dfDOE.dataX.max()-dfDOE.dataX.min()))
    
    Xmargin = 0.5
    
    for i, mcl in enumerate(np.sort(dfDOE['MCL'].unique())):            
        
        fig.add_trace(go.Carpet(
            a = dfDOE[dfDOE.MCL==mcl]['Total_Area'],
            b = dfDOE[dfDOE.MCL==mcl]['MTO_AET'],
            y = dfDOE[dfDOE.MCL==mcl][param],
            x = dfDOE[dfDOE.MCL==mcl].dataXnorm + i*(1+Xmargin), #dataXnorm + i*(1+Xmargin),
            #cheaterslope=chSlope,
            carpet="c_{}".format(mcl),
            aaxis = dict(
                #tickprefix = 'Sref = ',
                #ticksuffix = 'm²',
                smoothing = 0,
                minorgridcount = 4, #24,
                gridcolor = 'black',
                minorgridcolor = "rgb(230, 230, 230)",
                title = dict(
                    text="Total area (m²)",
                    offset =-10)
            ),
            baxis = dict(
                #tickprefix = 'MTO_AET = ',
                #ticksuffix = '',
                smoothing = 0,
                minorgridcount = 3, #24,
                gridcolor = 'black',
                minorgridcolor = "rgb(230, 230, 230)",
                title = dict(
                    text="MTO_AET (lbf)",
                    offset =-10) 
            )
        ))
        
        fig.add_annotation(x=0.5+ i*(1+Xmargin), y=dfDOE[dfDOE.MCL==mcl][param].min(),
            text="MCL: {} lbf".format(mcl),
            showarrow=False,
            yshift=-50)

        fig.add_trace(go.Scattercarpet(
            a = dfDOE[(dfDOE.MCL==mcl) & (dfDOE.status==True)]['Total_Area'],
            b = dfDOE[(dfDOE.MCL==mcl) & (dfDOE.status==True)]['MTO_AET'],
            carpet="c_{}".format(mcl),
            showlegend = False,
            mode = "markers",
            marker = dict(
                color = "rgb(102, 255, 51)",
                size = 5
                ),
            name = ''
        ))

        fig.add_trace(go.Scattercarpet(
            a = dfDOE[(dfDOE.MCL==mcl) & (dfDOE.status==False)]['Total_Area'],
            b = dfDOE[(dfDOE.MCL==mcl) & (dfDOE.status==False)]['MTO_AET'],
            carpet="c_{}".format(mcl),
            showlegend = False,
            mode = "markers",
            marker = dict(
                color = 'red',
                symbol = 'circle',
                size = 5
                ),
            name = ''
        ))

        if 'OPT_CONFIG' in dfDOE.columns : 
            fig.add_trace(go.Scattercarpet(
                a = dfDOE[dfDOE.MCL==mcl]['Total_Area'],
                b = dfDOE[dfDOE.MCL==mcl]['MTO_AET'],
                carpet="c_{}".format(mcl),
                showlegend = False,
                mode = "text",
                text = dfDOE['OPT_CONFIG'],
                textposition = "top right",
                name = ''
            ))

        if len(L)>0:
            col = []
            for l in L:
                try:
                    col.append(l[3])
                except:
                    col.append('blue')
            fig.add_trace(go.Scattercarpet(
                a = np.array([l[0] for l in L]),
                b = np.array([l[1] for l in L]),
                carpet="c_{}".format(l[2]),
                showlegend = False,
                mode = "markers",
                marker = dict(
                    color = col, #np.array([l[2] for l in L]),
                    symbol = "star-square-open-dot",
                    size = 12
                    ),
                name = ''
        )
            )


        for k in isoDict.keys():
            if i == 0:
                shwlgd = True
            else:
                shwlgd = False
            
            if isoDict[k]['color'] != False:
                isoTLAR = compute_isoTLARmcl(isoDict[k], mcl)
                fig.add_trace(go.Scattercarpet(
                    a = isoTLAR['Total_Area'],
                    b = isoTLAR['MTO_AET'],
                    carpet="c_{}".format(mcl),
                    showlegend = shwlgd,
                    mode = 'lines',
                    line = dict(
                      shape = 'spline',
                      smoothing = 1,
                      color = isoDict[k]['color'],
                      dash = isoDict[k]['linetype']
                    ),
                    name = isoDict[k]['TLAR'] + ': ' + str(isoDict[k]['TLARtgt'])
                  ))

    margin=(dfDOE[param].max()-dfDOE[param].min())/10
    
    fig.update_layout(go.Layout(
        width=1100,
        height=510,
        margin=dict(t=5, b=35, l=60, r=5),
        paper_bgcolor="LightSteelBlue",
        plot_bgcolor ='rgba(250,250,250,1)',
        yaxis = dict(
            visible=True,
            title = dict(
                text= param +' '+ unit),
            #nticks = 30 # to be improved,
            range=[math.floor((dfDOE[param].min()-1.8*margin)/10)*10, math.floor((dfDOE[param].max()+margin)/10)*10],
            gridcolor='rgba(220,220,220,0.8)',
            gridwidth=1.2,
            showspikes = True,
            spikethickness = 1,
            spikedash = "longdashdot",
            spikemode = "across",
            spikesnap = "cursor"
            ),
        xaxis = dict(
            visible=False),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01)
        ))
    if not (chart_title is None):
        fig.update_layout(go.Layout(
            title = dict(
                text='{}'.format(chart_title),
                y=0.97,
                x=0.5,
                xanchor= 'center',
                yanchor= 'top',
                )
        ))

    return fig

In [7]:
# if key contains tlar, it will be used to display green and red dot. Otherwise it will only be used to plot iso line
isoDict = {  
    'tlar1' : {
        'TLAR': 'TOFL_generic',
        'TLARtgt': 2400., # 1400.
        'crit': '<',
        'color': 'orange',
        'linetype': 'solid',
        'filter': {
            'by': 'TOFL_generic',
            'TLARtgt': 1.,
            'crit': '>'}
        },
    'tlar2' : {
        'TLAR': 'VLS',
        'TLARtgt': 140., #120.
        'crit': '<',
        'color': 'green',
        'linetype': 'solid'
        },
    'tlar3' : {
        'TLAR':'TTC',
        'TLARtgt': 25, # 0.3, 
        'crit': '<',
        'color': 'blue',
        'linetype': 'solid'
        },
    'tlar4' : {
        'TLAR':'ICAC',
        'TLARtgt':33000., # crit is 25000ft => 
        'crit': '>',
        'color': 'purple', # do not plot iso lines (e.g. because not a continuous value)
        'linetype': 'solid' # use to highlight low accuracy of curve due to the fact it is categorical variable (no interpoaltion)
        } #,
}
'''
    'trade1' : {
        'TLAR': 'TOFL',
        'TLARtgt': 1600.,
        'crit': '<',
        'color': 'orange',
        'linetype': 'dash'
        },
    'trade2' : {
        'TLAR': 'VAPP',
        'TLARtgt': 125.,
        'crit': '<',
        'color': 'green',
        'linetype': 'dash'
        },
    'trade3' : {
        'TLAR': 'Cruise_ratio',
        'TLARtgt': 0.25,
        'crit': '<',
        'color': 'blue',
        'linetype': 'dash',
        'filter': {
            'by': 'init_Cruise_Alt',
            'TLARtgt': 23001.,
            'crit': '>'}
        },
    'trade4' : {
        'TLAR': 'TOFL',
        'TLARtgt': 2000.,
        'crit': '<',
        'color': 'orange',
        'linetype': 'dot'
        },
        
}'''

"\n    'trade1' : {\n        'TLAR': 'TOFL',\n        'TLARtgt': 1600.,\n        'crit': '<',\n        'color': 'orange',\n        'linetype': 'dash'\n        },\n    'trade2' : {\n        'TLAR': 'VAPP',\n        'TLARtgt': 125.,\n        'crit': '<',\n        'color': 'green',\n        'linetype': 'dash'\n        },\n    'trade3' : {\n        'TLAR': 'Cruise_ratio',\n        'TLARtgt': 0.25,\n        'crit': '<',\n        'color': 'blue',\n        'linetype': 'dash',\n        'filter': {\n            'by': 'init_Cruise_Alt',\n            'TLARtgt': 23001.,\n            'crit': '>'}\n        },\n    'trade4' : {\n        'TLAR': 'TOFL',\n        'TLARtgt': 2000.,\n        'crit': '<',\n        'color': 'orange',\n        'linetype': 'dot'\n        },\n        \n}"

# Generic 800nm

In [8]:
DOE=pd.read_csv('example.csv', sep=',')
DOE=DOE[DOE.MCL==5800]
# MWE_delta'  'Drag_factor'  'SFC_Factor'
#DOE = DOE[(DOE.SFC_Factor==1.01)]
DOE['status'] = DOE.apply(TLARstatus, axis=1, dico=isoDict)

In [10]:
p = 'MTOW'
carpetplot(p, unit='[kg]', L = [[147.2, 30400]])
e= round((get_value_at(p, 30000*1.05, 150)/get_value_at(p, 30000, 150)-1)*100,1)
w= round((get_value_at(p, 30000, 150*1.05)/get_value_at(p, 30000, 150)-1)*100,1)
print("{} sensitivities:\n+5% eng => {}% \n+5% wing => {}%".format(p, e,w))

MTOW sensitivities:
+5% eng => 0.5% 
+5% wing => 0.5%


# MCL parameters

The purpose here is to check whether the optimum MCL thrust is influenced by the use cases. This can be done only considering fuel_cost as MCL effects on other costs (e.g. engine DMC) is not totally captured.

In [11]:
UC = 'Generic800'
DOE=pd.read_csv('example.csv'.format(UC), sep=',')
DOE['status'] = DOE.apply(TLARstatus, axis=1, dico=isoDict)
interp=getMultiLinInterp(DOE, ['Total_Area', "MTO_AET", "MCL"])

p = 'fuel_cost'
f = carpetplot3D(p, unit='[$]', L = [[148.5, 30550, 6600]], chSlope=1, chart_title=UC)
iplot(f)
mclref=6600
d=-5
clb= round((interp[p]([148.5, 30550, mclref*(1+d/100.)])/interp[p]([148.5, 30550, mclref])-1)*100,1)
print("{} sensitivities:\n@ {}lbf {}% MCL => {}%".format(p, mclref, d, clb))
mclref=5800
d=+5
clb= round((interp[p]([148.5, 30550, mclref*(1+d/100.)])/interp[p]([148.5, 30550, mclref])-1)*100,1)
print("@ {}lbf {}% MCL => {}%".format(mclref, d, clb))

/var/folders/cl/1j2m5n5d6tz7j370j52_b9v00000gn/T/ipykernel_74276/1155081908.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/cl/1j2m5n5d6tz7j370j52_b9v00000gn/T/ipykernel_74276/1155081908.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/to127534/opt/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning:

divide by zero encountered in true_divide



TypeError: type numpy.ndarray doesn't define __round__ method